In [1]:
from skimage.feature import hog
from skimage import io,color
from skimage.transform import resize
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics
import os
from  LBP_descriptor import LocalBinaryPatterns
import commonfunctions as cf
import cv2
import pandas as pd


# Get our training data 
X_train: features of training data.\
Y_train: labels of training data (1-->F, 0--> M).


In [38]:
Y_train=[]

files = os.listdir("Training_data/preprocessed/")

for file in files:
    if file[0]=='F':
        Y_train.append(1)
    else:
        Y_train.append(0)

Y_train=np.array(Y_train).astype(float)


# HOG feature

In [39]:
def HOG(img):
    img = np.array(resize(io.imread(img),(128,64))) 
    feature_vector, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(5,5), visualize=True)
    return feature_vector,hog_image


# LBP feature and histogram descriptor

In [5]:
# create object from LBP class to use it as our descriptor
# takes 2 parameters: number of data (train + test ) and number of neighbors
desc = LocalBinaryPatterns(258, 8)

# Extract features from training data

In [41]:
#to save your time: 
# run this cell once and the features will be saved in external file so that you can read them by running the next cell.
X_train=[]
files = os.listdir("Training_data/preprocessed/")

for file in files:

    #------------------- HOG feature------------------------
    #feature_vector,hog_image=HOG("Training_data/preprocessed/"+file)
    #--------------------------------------------------------
    #X_train.append(feature_vector)
    
    #------------------- LBP feature------------------------
    img = io.imread("Training_data/preprocessed/"+file )
    img = cf.downSize(img , 0.5)
    hist = desc.describe(img)
    X_train.append(hist)
    #--------------------------------------------------------
    
X_train=np.array(X_train)

#write feature vector of each image in external file
with open('LBP_descriptor.npy', 'wb') as f:
    np.save(f, X_train)
f.close()     

In [3]:
# read feature vector of training data from the npy file 
with open('LBP_descriptor.npy', 'rb') as f:
    X_train = np.load(f)
f.close()  

[[1.85356828e-02 4.79450421e-03 8.80866623e-04 ... 1.70380391e-06
  8.28336644e-01 1.27514389e-01]
 [1.32188627e-02 3.19657468e-03 6.18297702e-04 ... 0.00000000e+00
  8.90632179e-01 7.75707010e-02]
 [9.01382887e-03 2.23941227e-03 3.61279170e-04 ... 7.69230769e-05
  8.35508211e-01 1.00595506e-01]
 ...
 [9.31152050e-03 1.87066059e-03 4.61994671e-04 ... 0.00000000e+00
  9.18829805e-01 5.49511026e-02]
 [1.72780876e-02 3.27640293e-03 6.31753224e-04 ... 0.00000000e+00
  8.30305275e-01 1.16662310e-01]
 [2.34644685e-02 4.53319049e-03 1.07378090e-03 ... 0.00000000e+00
  8.10999683e-01 1.27926239e-01]]


# Get our test data 

In [6]:
#to save your time: 
# run this cell once and the features will be saved in external files so that you can read them by running the next cell.
X_test=[]
Y_test=[]

files = os.listdir("Test_data/preprocessed/")

for file in files:
    if file[0]=='F':
        Y_test.append(1)
    else:
        Y_test.append(0)
    #------------------- HOG feature------------------------
    #feature_vector,hog_image=HOG("our dataset/test/"+file)
    #X_test.append(feature_vector)
    
    #------------------- LBP feature------------------------
    img = io.imread("Test_data/preprocessed/" +file )
    img = cf.downSize(img , 0.5)
    hist = desc.describe(img)

    X_test.append(hist)
    #--------------------------------------------------------

    
Y_test=np.array(Y_test)
X_test=np.array(X_test)

#write feature vector of test data in external file
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)
f.close()   
#write labels of test data in external file
with open('Y_test.npy', 'wb') as f:
    np.save(f, Y_test)
f.close()     

In [4]:
# read feature vector of test data from the npy file 
with open('X_test.npy', 'rb') as f:
    X_test = np.load(f)
f.close()  
# read labels of test data from the npy file 
with open('Y_test.npy', 'rb') as f:
    Y_test = np.load(f)
f.close()  

# Random forest classifier

In [43]:

clf=RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")


Accuracy: 59.61538461538461 %


# Linear SVM classifier

In [44]:
clf=LinearSVC(C=100.0, random_state=42)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 71.15384615384616 %


c:\Users\dell\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


# Adaboost classifier

In [45]:
clf=AdaBoostClassifier(n_estimators=500)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 53.84615384615385 %


# KNN classifier

In [46]:
accuracies=[]
for k in range(1,30):
    clf=KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train,Y_train)
    Y_Predicted=clf.predict(X_test)
    accuracies.append(metrics.accuracy_score(Y_test, Y_Predicted)*100)

print("Accuracy:",accuracies[np.argmax(accuracies)],"%")

Accuracy: 61.53846153846154 %
